In [ ]:
from aiohttp import ClientSession
from cached_historical_data_fetcher import HistoricalDataCacheWithChunk
from lxml import html
from pandas import DataFrame, MultiIndex, Timestamp, Timedelta
from typing import Any

# from bs4 import BeautifulSoup

async def get_anime_list(session: ClientSession, date: Timestamp) -> DataFrame:
    """Get anime list from anime.eiga.com.
    This method is for testing purposes only. Do not use it in production.
    It seems to be prohibited to use this method for commercial purposes
    according to the terms of use of the site.
    See the terms of use for details: https://eiga.com/info/kiyaku/

    Parameters
    ----------
    session : ClientSession
        Session to use for the request.
    date : Timestamp
        Date to get anime list for.

    Returns
    -------
    DataFrame
        DataFrame with anime titles and dates.
    """
    seasons = ['spring', 'summer', 'fall', 'winter']
    season = seasons[date.quarter - 1]
    year = date.year
    
    url = f'https://anime.eiga.com/program/season/{year}-{season}/'
    resp = await session.get(url)
    text = await resp.text()
    htm = html.fromstring(text)
    # findi <a> with parent: <p class="seasonAnimeTtl">
    titles = htm.xpath('//p[@class="seasonAnimeTtl"]/a/text()')
    # format and print
    # print(BeautifulSoup(text, 'html.parser').prettify())
    # df = DataFrame(titles, columns=['title'], index=date_range(date, date + Timedelta(days=1), periods=len(titles)))
    df = DataFrame(titles, columns=['title'], index=MultiIndex.from_arrays([[date] * len(titles), range(len(titles))], names=['date', 'number']))
    return df

class AnimeList(HistoricalDataCacheWithChunk):
    """Anime list from anime.eiga.com.
    This class is for testing purposes only. Do not use it in production.
    It seems to be prohibited to use this method for commercial purposes
    according to the terms of use of the site.
    See the terms of use for details: https://eiga.com/info/kiyaku/"""
    delay_seconds = 1
    interval = Timedelta(days=365 // 4 + 1)
    start_init = Timestamp('2022-01-01', tz="Asia/Tokyo")
    
    async def __aenter__(self):
        self.session = ClientSession()
        return self
    
    async def __aexit__(self, *args):
        await self.session.close()
    
    async def get_one(self, date: Timestamp, *args: Any, **kwargs: Any) -> DataFrame:
        return await get_anime_list(self.session, date)
    
async with AnimeList() as anime_list:
    df = await anime_list.update(True, "args", kw="args")
    display(df)